In [1]:
import nltk
from nltk import word_tokenize,sent_tokenize
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split, cross_val_score 
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer

In [2]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Input
from keras.layers import Embedding 
from keras.layers import Conv1D, GlobalMaxPooling1D, GlobalMaxPool1D
from keras.layers import LSTM, Bidirectional

In [3]:
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences 
from tensorflow.keras import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers

In [4]:
dt = pd.read_csv('train.csv')

In [5]:
# english data set
df_en = dt.loc[dt['lang_abv'] == 'en']
df_en

,id,premise,hypothesis,lang_abv,language,label
0,5130fd2cb5,and these comments were considered in formulat...,The rules developed in the interim were put to...,en,English,0
1,5b72532a0b,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,en,English,2
3,5622f0c60b,you know they can't really defend themselves l...,They can't defend themselves because of their ...,en,English,0
7,fdcd1bd867,From Cockpit Country to St. Ann's Bay,From St. Ann's Bay to Cockpit Country.,en,English,2
8,7cfb3d272c,"Look, it's your skin, but you're going to be i...",The boss will fire you if he sees you slacking...,en,English,1
...,...,...,...,...,...,...
12115,2b78e2a914,The results of even the most well designed epi...,All studies have the same amount of uncertaint...,en,English,2
12116,7e9943d152,But there are two kinds of the pleasure of do...,But there are two kinds of the pleasure of doi...,en,English,0
12117,5085923e6c,The important thing is to realize that it's wa...,"It cannot be moved, now or ever.",en,English,2
12118,fc8e2fd1fe,At the west end is a detailed model of the who...,The model temple complex is at the east end.,en,English,2


In [6]:
#english x
x_en = dt.premise.values +' '+ dt.hypothesis.values
#english y 
y_en = dt.label.values
# split train and test
X_train_en, X_test_en, y_train_en, y_test_en = train_test_split(x_en, y_en, test_size=0.3, random_state=100)

In [7]:
X_train_en

array(['Υποθέτουμε μια άμεση γραμμική σχέση μεταξύ όγκου ανά κάτοικο και τεμαχίων ανά πιθανού Γνωρίζουμε πως δεν υπάρχει καμία δυνατή σύνδεση μεταξύ της ποσότητας ανά κάτοικο και κομματιών.',
       'Anstelle einer zentralisierten oder dezentralisierten CIO-Organisationen, führende Organisationen verwalten ihre Informationsressourcen durch eine Kombination solcher Strukturen. Organisationen verwalten ihre Informationsressourcen auf unterschiedlichen Wegen.',
       "and clean up is is uh is a joy uh a little soap and water and air dry them and you don't have to worry about that You don't need any soap for the clean up.",
       ...,
       'From here, many HIV researchers are putting their hopes on combining drug treatments with strategies that boost the immune system. Drug treatments combined with strategies that boost the immune system could be the solution to HIV.',
       'Thêm thời gian thông thường cần được thêm vào để phát triển kế hoạch hành động Họ thường cần thêm một năm để t

In [8]:
X_train_tok = [word_tokenize(sentence) for sentence in X_train_en]
X_test_tok = [word_tokenize(sentence) for sentence in X_test_en]

In [9]:
X_train_tok

[['Υποθέτουμε',
  'μια',
  'άμεση',
  'γραμμική',
  'σχέση',
  'μεταξύ',
  'όγκου',
  'ανά',
  'κάτοικο',
  'και',
  'τεμαχίων',
  'ανά',
  'πιθανού',
  'Γνωρίζουμε',
  'πως',
  'δεν',
  'υπάρχει',
  'καμία',
  'δυνατή',
  'σύνδεση',
  'μεταξύ',
  'της',
  'ποσότητας',
  'ανά',
  'κάτοικο',
  'και',
  'κομματιών',
  '.'],
 ['Anstelle',
  'einer',
  'zentralisierten',
  'oder',
  'dezentralisierten',
  'CIO-Organisationen',
  ',',
  'führende',
  'Organisationen',
  'verwalten',
  'ihre',
  'Informationsressourcen',
  'durch',
  'eine',
  'Kombination',
  'solcher',
  'Strukturen',
  '.',
  'Organisationen',
  'verwalten',
  'ihre',
  'Informationsressourcen',
  'auf',
  'unterschiedlichen',
  'Wegen',
  '.'],
 ['and',
  'clean',
  'up',
  'is',
  'is',
  'uh',
  'is',
  'a',
  'joy',
  'uh',
  'a',
  'little',
  'soap',
  'and',
  'water',
  'and',
  'air',
  'dry',
  'them',
  'and',
  'you',
  'do',
  "n't",
  'have',
  'to',
  'worry',
  'about',
  'that',
  'You',
  'do',
  "n't",


In [10]:
from gensim.models import Word2Vec
w2v_model = Word2Vec(X_train_tok, vector_size=100, window=10, min_count=5, sg=1, epochs=20, negative=10)

In [11]:
words = set(w2v_model.wv.index_to_key )
X_train_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words],dtype=object)
                         for ls in X_train_tok],dtype=object)
X_test_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words],dtype=object)
                         for ls in X_test_tok],dtype=object)

In [12]:
X_train_vect

array([array([[-0.3613430857658386, -0.15487268567085266, 0.6210166215896606,
               ..., -0.492960661649704, -0.3273857831954956,
               0.36922040581703186],
              [-0.20187164843082428, -0.059087835252285004, 0.30983710289001465,
               ..., -0.19016431272029877, -0.24204526841640472,
               0.3491160571575165],
              [-0.3763562738895416, -0.01369821559637785, 0.38828742504119873,
               ..., -0.23246288299560547, -0.31832417845726013,
               0.31041228771209717],
              ...,
              [-0.3763562738895416, -0.01369821559637785, 0.38828742504119873,
               ..., -0.23246288299560547, -0.31832417845726013,
               0.31041228771209717],
              [-0.31364116072654724, -0.24295388162136078, 0.44481024146080017,
               ..., -0.3532246947288513, -0.6034913659095764,
               0.4184260368347168],
              [-0.11495465785264969, 0.09611905366182327, 0.12984953820705414,
       

In [13]:
# Compute sentence vectors by averaging the word vectors for the words contained in the sentence
X_train_vect_avg = []
for v in X_train_vect:
    if v.size:
        X_train_vect_avg.append(v.mean(axis=0))
    else:
        X_train_vect_avg.append(np.zeros(100, dtype=float))
        
X_test_vect_avg = []
for v in X_test_vect:
    if v.size:
        X_test_vect_avg.append(v.mean(axis=0))
    else:
        X_test_vect_avg.append(np.zeros(100, dtype=float))

In [14]:
# data preprocessing
from sklearn.preprocessing import MinMaxScaler, Normalizer, StandardScaler

scaler =  MinMaxScaler()
scaler.fit(X_train_vect_avg)
X_train_en = scaler.transform(X_train_vect_avg)
X_test_en = scaler.transform(X_test_vect_avg)

In [15]:
# Naive Bayes Classifier
NBC_pipeline = Pipeline([
    ('sel', SelectKBest(chi2, k= 15)),  # feature selection
    ('learner', MultinomialNB())  # learning algorithm
])

NBC_pipeline.fit(X_train_en,y_train_en)
predictions = NBC_pipeline.predict(X_test_en)

print('Classification report:')
print(classification_report(y_test_en, predictions))
print('Confusion matrix:')
cm = confusion_matrix(y_test_en, predictions)
print(cm)

Classification report:
              precision    recall  f1-score   support

           0       0.34      0.99      0.51      1245
           1       0.00      0.00      0.00      1161
           2       0.39      0.01      0.02      1230

    accuracy                           0.34      3636
   macro avg       0.24      0.33      0.18      3636
weighted avg       0.25      0.34      0.18      3636

Confusion matrix:
[[1231    0   14]
 [1156    0    5]
 [1218    0   12]]


C:\Users\eleon\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\eleon\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\eleon\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [16]:
#SVM
param_grid = [{'sel__k': [10, 15, 20, 50], 'learner__C': [0.1, 1, 10,15]}]

opt_pipeline = Pipeline([
    
    ('sel', SelectKBest(chi2)),  # feature selection
    ('tfidf', TfidfTransformer()),  # weighting
    ('learner', LinearSVC())  # learning algorithm
])

n_jobs = 3 #Number of jobs to run in parallel
SVM_opt_search = GridSearchCV(opt_pipeline, param_grid, n_jobs = n_jobs, verbose=True).fit(X_train_en,y_train_en)
SVM_opt_search.best_params_

Fitting 5 folds for each of 16 candidates, totalling 80 fits


{'learner__C': 0.1, 'sel__k': 50}

In [17]:
opt_predictions = SVM_opt_search.best_estimator_.predict(X_test_en)

print('Classification report:')
print(classification_report(y_test_en, opt_predictions))
print('Confusion matrix:')
cm = confusion_matrix(y_test_en, opt_predictions)
print(cm)

Classification report:
              precision    recall  f1-score   support

           0       0.34      0.61      0.43      1245
           1       0.29      0.08      0.13      1161
           2       0.33      0.28      0.30      1230

    accuracy                           0.33      3636
   macro avg       0.32      0.32      0.29      3636
weighted avg       0.32      0.33      0.29      3636

Confusion matrix:
[[760 122 363]
 [729  98 334]
 [769 120 341]]


In [18]:
# Random Forest
param_grid = [{'sel__k': [8, 10, 5], 'learner__n_estimators': [100, 50, 500]}]

opt_pipeline = Pipeline([
    ('sel', SelectKBest(chi2)),  # feature selection
    ('tfidf', TfidfTransformer()),  # weighting
    ('learner', RandomForestClassifier(bootstrap = True))  # learning algorithm
])

n_jobs = 3 #Number of jobs to run in parallel
opt_search = GridSearchCV(opt_pipeline, param_grid, cv=5, n_jobs = n_jobs, verbose=True).fit(X_train_en,y_train_en)
opt_search.best_params_

Fitting 5 folds for each of 9 candidates, totalling 45 fits


{'learner__n_estimators': 50, 'sel__k': 5}

In [19]:
opt_predictions = opt_search.best_estimator_.predict(X_test_en)

print('Classification report:')
print(classification_report(y_test_en, opt_predictions))
print('Confusion matrix:')
cm = confusion_matrix(y_test_en, opt_predictions)
print(cm)

Classification report:
              precision    recall  f1-score   support

           0       0.34      0.35      0.35      1245
           1       0.31      0.33      0.32      1161
           2       0.31      0.28      0.30      1230

    accuracy                           0.32      3636
   macro avg       0.32      0.32      0.32      3636
weighted avg       0.32      0.32      0.32      3636

Confusion matrix:
[[437 414 394]
 [394 388 379]
 [450 431 349]]


## neural network

In [53]:
word_vectors = w2v_model.wv

In [55]:
MAX_NB_WORDS = len(word_vectors.key_to_index)
MAX_SEQUENCE_LENGTH = 200

In [ ]:
from keras.preprocessing.sequence import pad_sequences
